In [ ]:
#pip install ipywidgets

In [4]:
import requests
import pandas as pd
import time

In [5]:
f = open("token_TMDB.txt","r")
token = f.read()
headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}

In [6]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

In [7]:
res = requests.get(url, headers=headers).json()
res

{'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 16, 'name': 'Animation'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 80, 'name': 'Crime'},
  {'id': 99, 'name': 'Documentary'},
  {'id': 18, 'name': 'Drama'},
  {'id': 10751, 'name': 'Family'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 36, 'name': 'History'},
  {'id': 27, 'name': 'Horror'},
  {'id': 10402, 'name': 'Music'},
  {'id': 9648, 'name': 'Mystery'},
  {'id': 10749, 'name': 'Romance'},
  {'id': 878, 'name': 'Science Fiction'},
  {'id': 10770, 'name': 'TV Movie'},
  {'id': 53, 'name': 'Thriller'},
  {'id': 10752, 'name': 'War'},
  {'id': 37, 'name': 'Western'}]}

In [8]:
genres = pd.DataFrame(res['genres'])

In [9]:
genres.to_csv('genres.csv', index=False)

In [7]:
url = "https://api.themoviedb.org/3/movie/27205/credits?language=en-US"
headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}
response = requests.get(url, headers=headers)

In [8]:
data = response.json()

In [10]:
directors = pd.DataFrame([member for member in data['crew'] if member.get('job', '') == 'Director'])

In [11]:
directors

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job
0,False,2,525,Directing,Christopher Nolan,Christopher Nolan,76.031,/xuAIuYSmsUzKlUMBFGVZaWsY3DZ.jpg,5e83ac2ee33f830018359a00,Directing,Director


In [24]:
url = "https://api.themoviedb.org/3/movie/187&?append_to_response=keywords,credits"
headers = {"accept": "application/json", "Authorization": f"Bearer {token}"}
res2 = requests.get(url, headers=headers)

In [25]:
a = res2.json()

In [28]:
a['production_countries']

[{'iso_3166_1': 'US', 'name': 'United States of America'}]

In [ ]:
def get_url(imdb_id):
    url = f"https://api.themoviedb.org/3/find/{imdb_id}?external_source=imdb_id"
    try:
        res = requests.get(url, headers=headers).json()
        results = res.get('movie_results', [])
        if results:
            tmdb_id = results[0]['id']
            api_url = f"https://api.themoviedb.org/3/movie/{tmdb_id}?append_to_response=keywords,credits"
            try:
                response = requests.get(api_url, headers=headers)
                print(response)
                return response
            
            except Exception as e:
                    print(f"Error processing URL {api_url}: {e}")
        else:
            print(f'The movie {imdb_id} is not found in the TMDB')
    
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
                


In [ ]:
format_movie_details_imdb(get_url('tt0479884'))

In [ ]:
def find_directors(crew_list):
    directors = set()
    for crew_member in crew_list:
        if crew_member.get('job') == 'Director':
            directors.add(crew_member['name'])
    if not directors:
        return None
    else:
        return directors

In [ ]:
def find_writers(crew_list):
    writers = set()
    for crew_member in crew_list:
        if crew_member.get('job') == 'Writer':
            writers.add(crew_member['name'])
    if not writers:
        return None
    else:
        return writers

In [ ]:
def format_movie_details_imdb(response):
    data = response.json()
    details = {
        'tmdb_id': data['id'],
        'tconst': data['imdb_id'],
        'averageRating': data['vote_average'],
        'numVotes': data['vote_count'],
        'primaryTitle': data['title'],
        'originalTitle': data['original_title'],
        'directors': find_directors(data['credits']['crew']),
        'writers': find_writers(data['credits']['crew']),
        'adult': data['adult'],
        'startYear': data['release_date'].split('-')[0],
        'runtime': data['runtime'],
        'genres': set(genre['name'] for genre in data['genres']),
        'popularity' : data['popularity'],
        'budget': data['budget'],
        'revenue': data['revenue'],
        'overview': data['overview'],
        'poster_path': data['poster_path']}
    df = pd.DataFrame([details])
    
    return df

In [ ]:
%%time
x = format_movie_details_imdb(res2)
x

In [69]:
def format_movie_details_tmdb(response):
    data = response.json()
    relevant_data = {
        'budget': data['budget'],
        'genres': data['genres'],
        'homepage': data['homepage'],
        'id': data['id'],
        'keywords': data['keywords']['keywords'],
        'original_language': data['original_language'],
        'original_title': data['original_title'],
        'overview': data['overview'],
        'popularity': data['popularity'],
        'production_countries': data['production_countries'],
        'production_companies': data['production_companies'],
        'release_date': data['release_date'],
        'revenue': data['revenue'],
        'spoken_languages': data['spoken_languages'],
        'status': data['status'],
        'tagline': data['tagline'],
        'title': data['title'],
        'cast': data['credits']['cast'],
        'crew': data['credits']['crew']
    }

    df = pd.DataFrame([relevant_data])
    return df

In [98]:
def fetch_movie_details(series, header):
    df_combined = pd.DataFrame()  
    
    total_movies = len(series)
    movies_fetched = 0
    prev_progress = 0

    for movie_id in series:
        url = f"https://api.themoviedb.org/3/movie/{movie_id}?&append_to_response=keywords,credits"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            movie_df = format_movie_details_tmdb(response)
            df_combined = pd.concat([df_combined, movie_df], ignore_index=True)  
            movies_fetched += 1
        else:
            print(f"Failed to fetch details for movie with ID: {movie_id}")
        
        progress = (movies_fetched / total_movies) * 100
        if progress != prev_progress:
            print(f"Progress: {progress:.2f}% loading...", end="\r")  
            prev_progress = progress
        
        time.sleep(1/50)
        
    display(df_combined)
    return df_combined

In [99]:
final = pd.read_csv('final.csv')

In [100]:
final.drop(columns='Unnamed: 0', inplace=True)

In [3]:
a=final['id']

NameError: name 'final' is not defined

In [102]:
api_data = fetch_movie_details(a,headers)

Failed to fetch details for movie with ID: 2979
Failed to fetch details for movie with ID: 1193657


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_countries,production_companies,release_date,revenue,spoken_languages,status,tagline,title,cast,crew
0,160000000,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",https://www.warnerbros.com/movies/inception,27205,"[{'id': 10084, 'name': 'rescue'}, {'id': 10364...",en,Inception,"Cobb, a skilled thief who commits corporate es...",134.807,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 923, 'logo_path': '/8M99Dkt23MjQMTTWuk...",2010-07-15,825532764,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Your mind is the scene of the crime.,Inception,"[{'adult': False, 'gender': 2, 'id': 6193, 'kn...","[{'adult': False, 'gender': 2, 'id': 947, 'kno..."
1,165000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",http://www.interstellarmovie.net/,157336,"[{'id': 10084, 'name': 'rescue'}, {'id': 2964,...",en,Interstellar,The adventures of a group of explorers who mak...,224.000,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 923, 'logo_path': '/8M99Dkt23MjQMTTWuk...",2014-11-05,701729206,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Mankind was born on Earth. It was never meant ...,Interstellar,"[{'adult': False, 'gender': 2, 'id': 10297, 'k...","[{'adult': False, 'gender': 2, 'id': 947, 'kno..."
2,185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",https://www.warnerbros.com/movies/dark-knight/,155,"[{'id': 9537, 'name': 'joker'}, {'id': 4426, '...",en,The Dark Knight,Batman raises the stakes in his war on crime. ...,112.271,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 429, 'logo_path': '/2Tc1P3Ac8M479naPp1...",2008-07-16,1004558444,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Welcome to a world without rules.,The Dark Knight,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1113, 'kn..."
3,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.avatar.com/movies/avatar,19995,"[{'id': 2964, 'name': 'future'}, {'id': 3679, ...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",139.551,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 444, 'logo_path': None, 'name': 'Dune ...",2009-12-15,2923706026,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Enter the world of Pandora.,Avatar,"[{'adult': False, 'gender': 2, 'id': 65731, 'k...","[{'adult': False, 'gender': 2, 'id': 1721, 'kn..."
4,220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,"[{'id': 242, 'name': 'new york city'}, {'id': ...",en,The Avengers,When an unexpected enemy emerges and threatens...,129.160,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...",2012-04-25,1518815515,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,"[{'adult': False, 'gender': 2, 'id': 3223, 'kn...","[{'adult': False, 'gender': 2, 'id': 37, 'know..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37445,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,681169,[],pa,Galwakdi,"Jagteshwar, a librarian, has an obsessive-comp...",3.487,"[{'iso_3166_1': 'IN', 'name': 'India'}]","[{'id': 124796, 'logo_path': None, 'name': 'Om...",2022-04-08,0,"[{'english_name': 'Punjabi', 'iso_639_1': 'pa'...",Released,,Galwakdi,"[{'adult': False, 'gender': 2, 'id': 1919374, ...","[{'adult': False, 'gender': 1, 'id': 1817349, ..."
37446,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,714889,[],es,Los delincuentes,Morán works as a clerk in a bank in Buenos Air...,9.457,"[{'iso_3166_1': 'AR', 'name': 'Argentina'}, {'...","[{'id': 107826, 'logo_path': '/a0ujPI9vDmg41Ox...",2023-10-16,51000,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,Steal back your life.,The Delinquents,"[{'adult': False, 'gender': 2, 'id': 1489621, ...","[{'adult':

In [114]:
for index, row in final.iterrows():

    mask = api_data['id'] == row['id']
    if mask.any():
        api_data.loc[mask, 'vote_average'] = row['vote_average']
        api_data.loc[mask, 'vote_count'] = row['vote_count']
        api_data.loc[mask, 'runtime'] = row['runtime']


In [115]:
api_data.isna().sum()

budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_countries    0
production_companies    0
release_date            0
revenue                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
cast                    0
crew                    0
vote_average            0
vote_count              0
runtime                 0
dtype: int64

In [124]:
missing_ids = a[~a['id'].isin(api_data['id'])]

In [126]:
if not missing_ids.empty:
    api_data = pd.concat([api_data, missing_ids], ignore_index=True)
    api_data.reset_index(drop=True, inplace=True)


Added missing IDs to 'api_data'
            budget                                             genres  \
0      160000000.0  [{'id': 28, 'name': 'Action'}, {'id': 878, 'na...   
1      165000000.0  [{'id': 12, 'name': 'Adventure'}, {'id': 18, '...   
2      185000000.0  [{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...   
3      237000000.0  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
4      220000000.0  [{'id': 878, 'name': 'Science Fiction'}, {'id'...   
...            ...                                                ...   
37835          NaN  [{"id": 53, "name": "Thriller"}, {"id": 27, "n...   
37836          NaN                      [{"id": 18, "name": "Drama"}]   
37837          NaN  [{"id": 10769, "name": "Foreign"}, {"id": 53, ...   
37838          NaN  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
37839          NaN                [{"id": 99, "name": "Documentary"}]   

                                             homepage      id  \
0         https://www.warn

In [135]:
a.columns

Index(['id', 'genres', 'title', 'keywords', 'overview', 'popularity',
       'release_date', 'runtime', 'vote_average', 'cast', 'crew',
       'production_countries', 'production_companies'],
      dtype='object')

In [136]:
api_data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_countries',
       'production_companies', 'release_date', 'revenue', 'spoken_languages',
       'status', 'tagline', 'title', 'cast', 'crew', 'vote_average',
       'vote_count', 'runtime'],
      dtype='object')

In [137]:
data = api_data[['id','genres','title','keywords','overview',
           'popularity','release_date','runtime','vote_average',
           'cast','crew','production_countries','production_companies']]

In [138]:
data

,id,genres,title,keywords,overview,popularity,release_date,runtime,vote_average,cast,crew,production_countries,production_companies
0,27205,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",Inception,"[{'id': 10084, 'name': 'rescue'}, {'id': 10364...","Cobb, a skilled thief who commits corporate es...",134.807000,2010-07-15,148.0,8.8,"[{'adult': False, 'gender': 2, 'id': 6193, 'kn...","[{'adult': False, 'gender': 2, 'id': 947, 'kno...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 923, 'logo_path': '/8M99Dkt23MjQMTTWuk..."
1,157336,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",Interstellar,"[{'id': 10084, 'name': 'rescue'}, {'id': 2964,...",The adventures of a group of explorers who mak...,224.000000,2014-11-05,169.0,8.7,"[{'adult': False, 'gender': 2, 'id': 10297, 'k...","[{'adult': False, 'gender': 2, 'id': 947, 'kno...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 923, 'logo_path': '/8M99Dkt23MjQMTTWuk..."
2,155,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",The Dark Knight,"[{'id': 9537, 'name': 'joker'}, {'id': 4426, '...",Batman raises the stakes in his war on crime. ...,112.271000,2008-07-16,152.0,9.0,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1113, 'kn...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 429, 'logo_path': '/2Tc1P3Ac8M479naPp1..."
3,19995,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Avatar,"[{'id': 2964, 'name': 'future'}, {'id': 3679, ...","In the 22nd century, a paraplegic Marine is di...",139.551000,2009-12-15,162.0,7.9,"[{'adult': False, 'gender': 2, 'id': 65731, 'k...","[{'adult': False, 'gender': 2, 'id': 1721, 'kn...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 444, 'logo_path': None, 'name': 'Dune ..."
4,24428,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",The Avengers,"[{'id': 242, 'name': 'new york city'}, {'id': ...",When an unexpected enemy emerges and threatens...,129.160000,2012-04-25,143.0,8.0,"[{'adult': False, 'gender': 2, 'id': 3223, 'kn...","[{'adult': False, 'gender': 2, 'id': 37, 'know...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37835,286939,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 27, ""n...",Sanctuary: Quite a Conundrum,[],"It should have been just a normal day of sex, ...",0.166513,2012,82.0,0.0,"[{""cast_id"": 3, ""character"": ""Mimi"", ""credit_i...","[{""credit_id"": ""545f8107c3a3686cbb0041fd"", ""de...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""name"": ""Gold Lion Films"", ""id"": 37870}, {""n..."
37836,124606,"[{""id"": 18, ""name"": ""Drama""}]",Bang,"[{""id"": 10726, ""name"": ""gang""}, {""id"": 33928, ...",A young woman in L.A. is having a bad day: she...,0.918116,1995,98.0,6.0,"[{""cast_id"": 2, ""character"": ""The Girl"", ""cred...","[{""credit_id"": ""52fe4ab0c3a368484e161add"", ""de...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""name"": ""Asylum Films"", ""id"": 10571}, {""name..."
37837,67238,"[{""id"": 10769, ""name"": ""Foreign""}, {""id"": 53, ...",Cavite,[],"Adam, a security guard, travels from Californi...",0.022173,2005,80.0,7.5,[],"[{""credit_id"": ""52fe475dc3a368484e0c319f"", ""de...",[],[]
37838,231617,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","Signed, Sealed, Delivered","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013,120.0,7.0,"[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""name"": ""Front Street Pictures"", ""id"": 3958}..."


In [139]:
#data.to_csv('main_api.csv',index=False)

In [142]:
data = pd.read_csv('main_api.csv')

In [143]:
data['release_date'] = data['release_date'].apply(lambda x: x.split('-')[0] if isinstance(x, str) else x)

In [147]:
data.isna().sum()

id                      0
genres                  0
title                   0
keywords                0
overview                0
popularity              0
release_date            0
runtime                 0
vote_average            0
cast                    0
crew                    0
production_countries    0
production_companies    0
dtype: int64